In [1]:
from __future__ import annotations

import numpy as np
import jax.numpy as jnp
import pandas as pd
import polars as pl
import seaborn as sns
from scipy.stats import binom
from nptyping import NDArray, Shape, Int, Float, Bool
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from rich import print


def load_mathjax():
    """Load MathJax library in JupyterLab to enable LaTeX rendering in Plotly charts.

    This function checks if the MathJax library is already loaded, and if not,
    it loads the library from the provided CDN link.
    """
    display(HTML("""
        <script>
            if (typeof MathJax === 'undefined') {
                var script = document.createElement('script');
                script.type = 'text/javascript';
                script.src = 'https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-AMS-MML_HTMLorMML';
                document.head.appendChild(script);
            }
        </script>
    """))
# Load MathJax
load_mathjax()

In [4]:
def exponentially_weighted_average(y, beta):
    """Compute the exponentially weighted average of a given sequence.

    Parameters
    ----------
    y : numpy.ndarray
        A 1D numpy array of input values.
    beta : float
        The exponential decay factor, between 0 and 1.

    Returns
    -------
    numpy.ndarray
        A 1D numpy array of exponentially weighted averages.
    """
    n = len(y)
    averages = np.zeros(n)
    average = 0
    for i in range(n):
        average = beta * average + (1 - beta) * y[i]
        averages[i] = average
    return averages


def debiased_exponentially_weighted_average(y, beta):
    """Compute the debiased exponentially weighted average of a given sequence.

    Parameters
    ----------
    y : numpy.ndarray
        A 1D numpy array of input values.
    beta : float
        The exponential decay factor, between 0 and 1.

    Returns
    -------
    numpy.ndarray
        A 1D numpy array of debiased exponentially weighted averages.
    """
    n = len(y)
    averages = np.zeros(n)
    average = 0
    for i in range(n):
        average = beta * average + (1 - beta) * y[i]
        debiased_average = average / (1 - beta ** (i + 1))
        averages[i] = debiased_average
    return averages


def plot_averages(x, y, betas):
    """Plot the input data and the exponentially weighted averages 
    with and without bias correction for the given beta values.

    Parameters
    ----------
    x : numpy.ndarray
        A 1D numpy array of x-values for the input data.
    y : numpy.ndarray
        A 1D numpy array of y-values for the input data.
    betas : list of float
        A list of beta values for the exponential decay factor.
    """
    for beta in betas:
        df = pd.DataFrame({
            "x": x, 
            "y": y,
            "EMA": exponentially_weighted_average(y, beta),
            "EMA with bias correction": debiased_exponentially_weighted_average(y, beta)
        })
        fig = px.line(
            df, x="x", y=["y", "EMA", "EMA with bias correction"],
            title=f"beta = {beta:.2f}", 
            labels={"x": "x", "value": "y"})
        fig.show()


np.random.seed(0)
n = 50
x = np.arange(n) * np.pi
y = np.cos(x) * np.exp(x / 100) - 10 * np.exp(-0.01 * x)

betas = [0.9, 0.99]
plot_averages(x, y, betas)
